# Exploring Time Series data


## What is a trend ?

The trend component of a time series represents a persistent, long-term change in the mean of the series. The trend is the slowest-moving part of a series, the part representing the largest time scale of importance. In a time series of product sales, an increasing trend might be the effect of a market expansion as more people become aware of the product year by year.

## What is stationarity ? 
A stationary time series is one whose properties do not depend on the time at which the series is observed. 15. Thus, time series with trends, or with seasonality, are not stationary — the trend and seasonality will affect the value of the time series at different times.

## What is non-stationarity ?
A stationary time series has a constant variance and it always returns to the long-run mean. A time series whose statistical properties change over time is called a non-stationary time series. Thus a time series with a trend or seasonality is non-stationary in nature

## 1. Libraries and data paths

In [1]:

# data importing & processing
import os
import pandas as pd
from datetime import date

# dash & ploting
import dash
from dash import dcc, html, callback, Output, Input
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go

In [3]:
# declare paths
path_notebooks=os.path.abspath(os.path.join(os.getcwd(), os.pardir))
path_root = os.path.abspath(os.path.join(path_notebooks, os.pardir))
path_data = os.path.join(path_root, 'datafiles/datamarts', 'query002_rainfall.csv')

In [4]:
df_eda = pd.read_csv(path_data)
df_eda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2192 entries, 0 to 2191
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID_IAREA                     2192 non-null   int64  
 1   NAME                         2192 non-null   object 
 2   LATITUDE                     2192 non-null   float64
 3   LONGITUDE                    2192 non-null   float64
 4   COUNTRY                      2192 non-null   object 
 5   id_reg_nat                   2192 non-null   int64  
 6   time                         2192 non-null   object 
 7   temperature_2m_mean          2192 non-null   float64
 8   precipitation_sum            2192 non-null   float64
 9   wind_speed_10m_max           2192 non-null   float64
 10  wind_direction_10m_dominant  2192 non-null   int64  
 11  et0_fao_evapotranspiration   2192 non-null   float64
 12  id_area                      2192 non-null   int64  
dtypes: float64(6), int

## 2. Dataframe filter
Prefereble to filter the dataframe to make the plot overcramped with data 

In [9]:
df_eda.head(3)

,ID_IAREA,NAME,LATITUDE,LONGITUDE,COUNTRY,id_reg_nat,time,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,et0_fao_evapotranspiration,id_area
0,1,BHOPAL,23.25469,77.40289,INDIA,1,2020-11-19,23.3,1.2,14.8,255,2.84,1
1,1,BHOPAL,23.25469,77.40289,INDIA,2,2020-11-20,21.2,0.0,12.5,29,3.39,1
2,1,BHOPAL,23.25469,77.40289,INDIA,3,2020-11-21,18.4,0.0,10.2,45,3.64,1


In [10]:
df_eda.columns

Index(['ID_IAREA', 'NAME', 'LATITUDE', 'LONGITUDE', 'COUNTRY', 'id_reg_nat',
       'time', 'temperature_2m_mean', 'precipitation_sum',
       'wind_speed_10m_max', 'wind_direction_10m_dominant',
       'et0_fao_evapotranspiration', 'id_area'],
      dtype='object')

In [32]:
df_eda_f = df_eda[['time','temperature_2m_mean','precipitation_sum','COUNTRY','wind_speed_10m_max']]

In [33]:
# restrict on Algiers
df_eda_f_a = df_eda_f.query(" COUNTRY =='ALGERIA' ")

In [26]:
df_eda_f_a.head(3)

,time,temperature_2m_mean,precipitation_sum,COUNTRY
1096,2020-11-19,13.9,0.0,ALGERIA
1097,2020-11-20,13.5,8.0,ALGERIA
1098,2020-11-21,14.7,1.3,ALGERIA


In [37]:
def get_meteo_vars_fig(df):
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    
    fig.add_trace(go.Bar(
        x=df["time"],
        y=df["precipitation_sum"],
        name='precipitation_sum',
    ), secondary_y=False)

    fig.add_trace(go.Line(
        x=df["time"],
        y=df["temperature_2m_mean"],
        name='temperature_2m_mean',
    ), secondary_y=True)

    fig.add_trace(go.Line(
        x=df["time"],
        y=df["wind_speed_10m_max"],
        name='wind_speed_10m_max',
    ), secondary_y=True)

    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text']
    )
    fig.update_xaxes(
        rangeslider_visible=False,

        rangeselector=dict(
            buttons=
                list([
                dict(count=14, label="2w", step="day", stepmode="backward"),
                dict(count=30, label="1m", step="day", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(step="all",stepmode="backward"),
                #dict(count=1, label="1m", step="month", stepmode="backward"),
                ]

                )

        ))

    fig.update_xaxes(showgrid=True)
    #fig.update_xaxes(showgrid=True, tickfont_family="Arial Black")
    fig.update_yaxes()
    fig.update_layout(template="plotly_white")

    return fig


In [38]:
get_meteo_vars_fig(df_eda_f_a)

c:\Users\emanu\anaconda3\envs\hs01\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.





if you need to see graph without executing all the codes : 
see temp_precip.png in assets folder

# Correlation between Time Series 